# Marital status, Alcohol Consumption and Fatal Car Accidents

**Date:** 2021-11-29

**Reference:** M249, Book 1, Part 2

In [1]:
suppressPackageStartupMessages(library(tidyverse))
library(R249)
library(DescTools)

## Summary

## Get the data

In [2]:
(dat <- as_tibble(read.csv(file = "..\\..\\data\\drinkdriving.csv")))

count,level,exposure,outcome
<int>,<chr>,<chr>,<chr>
4,married,over 100mg,case
5,married,over 100mg,control
5,married,under 100mg,case
103,married,under 100mg,control
10,not married,over 100mg,case
3,not married,over 100mg,control
5,not married,under 100mg,case
43,not married,under 100mg,control


## Prepare the data

## Prepare the data

Cast the `exposure`, `outcome`, `level` columns to factors.

In [3]:
labexp <- c("under 100mg", "over 100mg")
labout <- c("control", "case")
lablev <- c("not married", "married")
(sorteddat <- dat %>%
    mutate(exposure = factor(dat$exposure, labexp)) %>%
    mutate(outcome = factor(dat$outcome, labout)) %>%
    mutate(level = factor(dat$level, lablev)) %>%
    arrange(level, exposure, outcome))

count,level,exposure,outcome
<int>,<fct>,<fct>,<fct>
43,not married,under 100mg,control
5,not married,under 100mg,case
3,not married,over 100mg,control
10,not married,over 100mg,case
103,married,under 100mg,control
5,married,under 100mg,case
5,married,over 100mg,control
4,married,over 100mg,case


Filter the tibble on each specific `level`, pull the `count` column as a vector and initilise a matrix.
Append this new matrix to an array.

In [4]:
notmarried <- filter(sorteddat, level == "not married") %>%
    pull(count) %>%
    matrix(nrow = 2, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout))
married <- filter(sorteddat, level == "married") %>%
    pull(count) %>%
    matrix(nrow = 2, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout))
datarr <- array(
    c(notmarried, married),
    dim = c(2, 2, 2),
    dimnames = list(labexp, labout, lablev)
)
print(datarr)

, , not married

            control case
under 100mg      43    5
over 100mg        3   10

, , married

            control case
under 100mg     103    5
over 100mg        5    4



## Stratum-specific odds ratios

In [5]:
# not married
oddsratio(datarr[, , 1])

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),28.66667,0.8103019,5.856619,140.3161


In [6]:
# married
oddsratio(datarr[, , 2])

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),16.48,0.8122246,3.354211,80.96998


In [7]:
oddsratio_crude(datarr)

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),25.55,0.5507067,8.682174,75.18883


## Tarone's test for homogeneity

In [8]:
BreslowDayTest(datarr, correct = TRUE)


	Breslow-Day Test on Homogeneity of Odds Ratios (with Tarone
	correction)

data:  datarr
X-squared = 0.23557, df = 1, p-value = 0.6274


## Mantel-Haenszel odds ratio and chi-squared test

In [9]:
mantelhaen.test(datarr)


	Mantel-Haenszel chi-squared test with continuity correction

data:  datarr
Mantel-Haenszel X-squared = 36.604, df = 1, p-value = 1.447e-09
alternative hypothesis: true common odds ratio is not equal to 1
95 percent confidence interval:
  7.465154 70.866332
sample estimates:
common odds ratio 
         23.00061 
